# Prediction Heart Disease Using Machine Learning 

This notebook showcases an end to end machine learning framework using data on heart disease from the University of California Irvine. 

## Problem definition

In a statement, 
> Given clinical parameters about a patient, can we predict whether or not a person has a heart disease?


## Evaluation
>If we can reach 95% accuracy at predicting whether or not a patient has heart disease in the proof of concept. We will pursue the project.

### Preparing tools

> Numpy

> Pandas

> Matplotlib

> Seaborn

> Scikit Learn

In [5]:
import numpy as np 
import pandas as pd 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve

### Exploring our Data